In [ ]:
# SCRAPPING CLIKANDBOAT - LA CORUÑA - JULIO 2026
# -*- coding: utf-8 -*-
"""
Scraping Click&Boat (La Coruña, 24–26 julio 2026, páginas 1–3)
Extrae: nombre del barco, tipo de barco, precio, zona, rating,
capacidad de personas, patrón (sí/no/opcional) y eslora (m).
"""

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd

# ------------------ CONFIGURACIÓN SELENIUM ------------------ #
options = webdriver.ChromeOptions()
options.add_argument("--headless")          # COMENTA esta línea si quieres ver el navegador
options.add_argument("--window-size=1920,1080")

driver = webdriver.Chrome(options=options)  # necesitas chromedriver en el PATH

# ------------------ PARÁMETROS DE BÚSQUEDA ------------------ #
URL_TEMPLATE = (
    "https://www.clickandboat.com/es/alquiler-barcos/busqueda"
    "?dateFrom=2026-07-24"
    "&dateTo=2026-07-26"
    "&where=Coru%C3%B1a,%20La%20Coru%C3%B1a,%20Espa%C3%B1a"
    "&page={page}"
)

BOAT_TYPES = {
    "Velero", "Lancha", "Catamarán", "Catamaran", "Neumática", "Neumatica",
    "Yate", "Moto de agua", "Goleta", "Casa flotante",
    "Barco sin licencia", "Bote sin licencia"
}

registros = []

# ------------------ RECORRER PÁGINAS ------------------ #
for page in range(1, 4):   # páginas 1, 2 y 3
    url = URL_TEMPLATE.format(page=page)
    print(f"\n=== PÁGINA {page}: {url} ===")
    driver.get(url)

    wait = WebDriverWait(driver, 25)

    try:
        # esperamos a que se carguen las tarjetas de producto
        wait.until(
            EC.presence_of_all_elements_located(
                (By.CSS_SELECTOR, "app-extended-boat-card.product__card.extendedBoatCard")
            )
        )
    except Exception as e:
        print(f"   No se encontraron tarjetas en la página {page}: {e}")
        continue

    cards = driver.find_elements(
        By.CSS_SELECTOR, "app-extended-boat-card.product__card.extendedBoatCard"
    )
    print(f"   Tarjetas encontradas: {len(cards)}")

    for card in cards:
        # --------- Nombre del barco --------- #
        try:
            nombre = card.find_element(
                By.CSS_SELECTOR, ".extendedBoatCard__label"
            ).text.strip()
        except Exception:
            nombre = ""

        # --------- Tipo de barco y patrón --------- #
        tipo = ""
        patron = ""
        try:
            highlights = card.find_elements(
                By.CSS_SELECTOR,
                "ul.extendedBoatCard__highlights li.highlight"
            )
            for h in highlights:
                txt = h.text.strip()
                if txt in BOAT_TYPES and not tipo:
                    tipo = txt
                if "patrón" in txt.lower():
                    patron = txt
        except Exception:
            pass

        # --------- Precio por día --------- #
        try:
            precio_raw = card.find_element(
                By.CSS_SELECTOR, "span.pricePanel__price"
            ).text
            precio = precio_raw.replace("\xa0", " ").strip()
        except Exception:
            precio = ""

        # --------- Zona / localización --------- #
        try:
            zona = card.find_element(
                By.CSS_SELECTOR, ".extendedBoatCard__localization"
            ).text.strip()
        except Exception:
            zona = ""

        # --------- Rating (span.rate__avg) --------- #
        try:
            rating_raw = card.find_element(
                By.CSS_SELECTOR, ".extendedBoatCard__rate span.rate__avg"
            ).text.strip()
            rating = rating_raw.replace(",", ".")
        except Exception:
            rating = ""

        # --------- Capacidad personas y eslora --------- #
        capacidad_personas = ""
        eslora_m = ""
        try:
            caps = card.find_elements(
                By.CSS_SELECTOR, ".extendedBoatCard__capacities span"
            )
            for c in caps:
                txt = c.text.strip()
                if "persona" in txt.lower():
                    capacidad_personas = txt      # ej. "8 personas"
                elif " m" in txt:
                    eslora_m = txt                # ej. "6 m"
        except Exception:
            pass

        registros.append(
            {
                "pagina": page,
                "nombre_barco": nombre,
                "tipo_barco": tipo,
                "precio_dia": precio,
                "zona": zona,
                "rating": rating,
                "capacidad_personas": capacidad_personas,
                "patron": patron,
                "eslora_m": eslora_m,
            }
        )

# cerramos navegador
driver.quit()

# ------------------ CREAR DATAFRAME Y GUARDAR ------------------ #
df = pd.DataFrame(
    registros,
    columns=[
        "pagina",
        "nombre_barco",
        "tipo_barco",
        "precio_dia",
        "zona",
        "rating",
        "capacidad_personas",
        "patron",
        "eslora_m",
    ]
)

print("\nColumnas del DataFrame:", df.columns.tolist())
print("Filas totales:", len(df))
print(df.head())

df.to_excel(
    "clickandboat_coruña_2026-07-24_26_final.xlsx",
    index=False
)
print("Excel guardado como 'clickandboat_coruña_2026-07-24_26_final.xlsx'")



=== PÁGINA 1: https://www.clickandboat.com/es/alquiler-barcos/busqueda?dateFrom=2026-07-24&dateTo=2026-07-26&where=Coru%C3%B1a,%20La%20Coru%C3%B1a,%20Espa%C3%B1a&page=1 ===
   Tarjetas encontradas: 18

=== PÁGINA 2: https://www.clickandboat.com/es/alquiler-barcos/busqueda?dateFrom=2026-07-24&dateTo=2026-07-26&where=Coru%C3%B1a,%20La%20Coru%C3%B1a,%20Espa%C3%B1a&page=2 ===
   Tarjetas encontradas: 18

=== PÁGINA 3: https://www.clickandboat.com/es/alquiler-barcos/busqueda?dateFrom=2026-07-24&dateTo=2026-07-26&where=Coru%C3%B1a,%20La%20Coru%C3%B1a,%20Espa%C3%B1a&page=3 ===
   Tarjetas encontradas: 18

Columnas del DataFrame: ['pagina', 'nombre_barco', 'tipo_barco', 'precio_dia', 'zona', 'rating', 'capacidad_personas', 'patron', 'eslora_m']
Filas totales: 54
   pagina                       nombre_barco tipo_barco precio_dia     zona  \
0       1          Quicksilver 600 Commander     Lancha      210 €  Riveira   
1       1      Quicksilver 525 Flamingo 75Hp     Lancha      175 €  Riveira 

In [ ]:
## SCRAPPING CLIKANDBOAT - MÁLAGA - JULIO 2026
#  -*- coding: utf-8 -*-
"""
Scraping Click&Boat (Málaga, 24–26 julio 2026, páginas 1–14)
Extrae: nombre del barco, tipo de barco, precio, zona, rating,
capacidad de personas, patrón (sí/no/opcional), eslora (m) y URL del anuncio.
"""

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd

# ------------------ CONFIGURACIÓN SELENIUM ------------------ #
options = webdriver.ChromeOptions()
options.add_argument("--headless")          # comenta esta línea si quieres ver el navegador
options.add_argument("--window-size=1920,1080")

driver = webdriver.Chrome(options=options)  # necesitas chromedriver en el PATH

# ------------------ PARÁMETROS DE BÚSQUEDA (MÁLAGA) ------------------ #
BASE_URL = (
    "https://www.clickandboat.com/es/alquiler-barcos/busqueda"
    "?dateFrom=2026-07-24"
    "&dateTo=2026-07-26"
    "&where=M%C3%A1laga,%20Espa%C3%B1a"
)
# Páginas:
# 1 -> BASE_URL
# 2..14 -> BASE_URL + "&_page={page}"

BOAT_TYPES = {
    "Velero", "Lancha", "Catamarán", "Catamaran", "Neumática", "Neumatica",
    "Yate", "Moto de agua", "Goleta", "Casa flotante",
    "Barco sin licencia", "Bote sin licencia"
}

registros = []

# ------------------ RECORRER PÁGINAS ------------------ #
for page in range(1, 15):  # 1 a 14
    if page == 1:
        url = BASE_URL
    else:
        url = BASE_URL + f"&_page={page}"

    print(f"\n=== PÁGINA {page}: {url} ===")
    driver.get(url)

    wait = WebDriverWait(driver, 20)

    try:
        wait.until(
            EC.presence_of_all_elements_located(
                (By.CSS_SELECTOR, "app-extended-boat-card.product__card.extendedBoatCard")
            )
        )
    except Exception as e:
        print(f"   No se encontraron tarjetas en la página {page}: {e}")
        continue

    cards = driver.find_elements(
        By.CSS_SELECTOR, "app-extended-boat-card.product__card.extendedBoatCard"
    )
    print(f"   Tarjetas encontradas: {len(cards)}")

    for card in cards:
        # --------- URL del anuncio --------- #
        try:
            link_el = card.find_element(
                By.CSS_SELECTOR, "a.extendedBoatCard__link"
            )
            url_barco = link_el.get_attribute("href")
        except Exception:
            url_barco = ""

        # --------- Nombre del barco --------- #
        try:
            nombre = card.find_element(
                By.CSS_SELECTOR, ".extendedBoatCard__label"
            ).text.strip()
        except Exception:
            nombre = ""

        # --------- Tipo de barco y patrón --------- #
        tipo = ""
        patron = ""
        try:
            highlights = card.find_elements(
                By.CSS_SELECTOR,
                "ul.extendedBoatCard__highlights li.highlight"
            )
            for h in highlights:
                txt = h.text.strip()
                if txt in BOAT_TYPES and not tipo:
                    tipo = txt
                if "patrón" in txt.lower():
                    patron = txt
        except Exception:
            pass

        # --------- Precio por día --------- #
        try:
            precio_raw = card.find_element(
                By.CSS_SELECTOR, "span.pricePanel__price"
            ).text
            precio = precio_raw.replace("\xa0", " ").strip()
        except Exception:
            precio = ""

        # --------- Zona / localización --------- #
        try:
            zona = card.find_element(
                By.CSS_SELECTOR, ".extendedBoatCard__localization"
            ).text.strip()
        except Exception:
            zona = ""

        # --------- Rating (span.rate__avg) --------- #
        try:
            rating_raw = card.find_element(
                By.CSS_SELECTOR, ".extendedBoatCard__rate span.rate__avg"
            ).text.strip()
            rating = rating_raw.replace(",", ".")
        except Exception:
            rating = ""

        # --------- Capacidad personas y eslora --------- #
        capacidad_personas = ""
        eslora_m = ""
        try:
            caps = card.find_elements(
                By.CSS_SELECTOR, ".extendedBoatCard__capacities span"
            )
            for c in caps:
                txt = c.text.strip()
                if "persona" in txt.lower():
                    capacidad_personas = txt
                elif " m" in txt:
                    eslora_m = txt
        except Exception:
            pass

        registros.append(
            {
                "pagina": page,
                "url_barco": url_barco,
                "nombre_barco": nombre,
                "tipo_barco": tipo,
                "precio_dia": precio,
                "zona": zona,
                "rating": rating,
                "capacidad_personas": capacidad_personas,
                "patron": patron,
                "eslora_m": eslora_m,
            }
        )

# cerramos navegador
driver.quit()

# ------------------ CREAR DATAFRAME, LIMPIAR DUPLICADOS Y GUARDAR ------------------ #
df = pd.DataFrame(registros)

print("\nFilas totales (con posibles duplicados):", len(df))

# Quitamos duplicados por URL de anuncio
if "url_barco" in df.columns:
    df = df.drop_duplicates(subset=["url_barco"])

print("Filas tras eliminar duplicados por URL:", len(df))
print(df.head())

df.to_excel(
    "clickandboat_malaga_2026-07-24_26_sin_duplicados.xlsx",
    index=False
)
print("Excel guardado como 'clickandboat_malaga_2026-07-24_26_sin_duplicados.xlsx'")



=== PÁGINA 1: https://www.clickandboat.com/es/alquiler-barcos/busqueda?dateFrom=2026-07-24&dateTo=2026-07-26&where=M%C3%A1laga,%20Espa%C3%B1a ===
   Tarjetas encontradas: 18

=== PÁGINA 2: https://www.clickandboat.com/es/alquiler-barcos/busqueda?dateFrom=2026-07-24&dateTo=2026-07-26&where=M%C3%A1laga,%20Espa%C3%B1a&_page=2 ===
   Tarjetas encontradas: 18

=== PÁGINA 3: https://www.clickandboat.com/es/alquiler-barcos/busqueda?dateFrom=2026-07-24&dateTo=2026-07-26&where=M%C3%A1laga,%20Espa%C3%B1a&_page=3 ===
   Tarjetas encontradas: 18

=== PÁGINA 4: https://www.clickandboat.com/es/alquiler-barcos/busqueda?dateFrom=2026-07-24&dateTo=2026-07-26&where=M%C3%A1laga,%20Espa%C3%B1a&_page=4 ===
   Tarjetas encontradas: 18

=== PÁGINA 5: https://www.clickandboat.com/es/alquiler-barcos/busqueda?dateFrom=2026-07-24&dateTo=2026-07-26&where=M%C3%A1laga,%20Espa%C3%B1a&_page=5 ===
   Tarjetas encontradas: 18

=== PÁGINA 6: https://www.clickandboat.com/es/alquiler-barcos/busqueda?dateFrom=2026-07-24&d

In [9]:
# -*- coding: utf-8 -*-
"""
Scraping Click&Boat (La Coruña, 24–26 julio 2026, páginas 1–3)
Extrae: nombre del barco, tipo de barco, precio, zona, rating,
capacidad de personas, patrón (sí/no/opcional), eslora (m) y URL del anuncio.
"""

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd

# ------------------ CONFIGURACIÓN SELENIUM ------------------ #
options = webdriver.ChromeOptions()
options.add_argument("--headless")          # comenta esta línea si quieres ver el navegador
options.add_argument("--window-size=1920,1080")

driver = webdriver.Chrome(options=options)  # necesitas chromedriver en el PATH

# ------------------ PARÁMETROS DE BÚSQUEDA (LA CORUÑA) ------------------ #
BASE_URL = (
    "https://www.clickandboat.com/es/alquiler-barcos/busqueda"
    "?where=La%20Coru%C3%B1a,%20Espa%C3%B1a"
    "&DateDebut=2026-07-24"
    "&DateFin=2026-07-26"
    "&ProduitTypeId="
    "&withCaptain="
)
# Páginas:
# 1 -> BASE_URL
# 2,3 -> BASE_URL + "&_page={page}"

BOAT_TYPES = {
    "Velero", "Lancha", "Catamarán", "Catamaran", "Neumática", "Neumatica",
    "Yate", "Moto de agua", "Goleta", "Casa flotante",
    "Barco sin licencia", "Bote sin licencia"
}

registros = []

# ------------------ RECORRER PÁGINAS ------------------ #
for page in range(1, 4):  # 1, 2, 3
    if page == 1:
        url = BASE_URL
    else:
        url = BASE_URL + f"&_page={page}"

    print(f"\n=== PÁGINA {page}: {url} ===")
    driver.get(url)

    wait = WebDriverWait(driver, 20)

    try:
        wait.until(
            EC.presence_of_all_elements_located(
                (By.CSS_SELECTOR, "app-extended-boat-card.product__card.extendedBoatCard")
            )
        )
    except Exception as e:
        print(f"   No se encontraron tarjetas en la página {page}: {e}")
        continue

    cards = driver.find_elements(
        By.CSS_SELECTOR, "app-extended-boat-card.product__card.extendedBoatCard"
    )
    print(f"   Tarjetas encontradas: {len(cards)}")

    for card in cards:
        # --------- URL del anuncio --------- #
        try:
            link_el = card.find_element(
                By.CSS_SELECTOR, "a.extendedBoatCard__link"
            )
            url_barco = link_el.get_attribute("href")
        except Exception:
            url_barco = ""

        # --------- Nombre del barco --------- #
        try:
            nombre = card.find_element(
                By.CSS_SELECTOR, ".extendedBoatCard__label"
            ).text.strip()
        except Exception:
            nombre = ""

        # --------- Tipo de barco y patrón --------- #
        tipo = ""
        patron = ""
        try:
            highlights = card.find_elements(
                By.CSS_SELECTOR,
                "ul.extendedBoatCard__highlights li.highlight"
            )
            for h in highlights:
                txt = h.text.strip()
                if txt in BOAT_TYPES and not tipo:
                    tipo = txt
                if "patrón" in txt.lower():
                    patron = txt
        except Exception:
            pass

        # --------- Precio por día --------- #
        try:
            precio_raw = card.find_element(
                By.CSS_SELECTOR, "span.pricePanel__price"
            ).text
            precio = precio_raw.replace("\xa0", " ").strip()
        except Exception:
            precio = ""

        # --------- Zona / localización --------- #
        try:
            zona = card.find_element(
                By.CSS_SELECTOR, ".extendedBoatCard__localization"
            ).text.strip()
        except Exception:
            zona = ""

        # --------- Rating (span.rate__avg) --------- #
        try:
            rating_raw = card.find_element(
                By.CSS_SELECTOR, ".extendedBoatCard__rate span.rate__avg"
            ).text.strip()
            rating = rating_raw.replace(",", ".")
        except Exception:
            rating = ""

        # --------- Capacidad personas y eslora --------- #
        capacidad_personas = ""
        eslora_m = ""
        try:
            caps = card.find_elements(
                By.CSS_SELECTOR, ".extendedBoatCard__capacities span"
            )
            for c in caps:
                txt = c.text.strip()
                if "persona" in txt.lower():
                    capacidad_personas = txt
                elif " m" in txt:
                    eslora_m = txt
        except Exception:
            pass

        registros.append(
            {
                "pagina": page,
                "url_barco": url_barco,
                "nombre_barco": nombre,
                "tipo_barco": tipo,
                "precio_dia": precio,
                "zona": zona,
                "rating": rating,
                "capacidad_personas": capacidad_personas,
                "patron": patron,
                "eslora_m": eslora_m,
            }
        )

# cerramos navegador
driver.quit()

# ------------------ CREAR DATAFRAME, QUITAR DUPLICADOS Y GUARDAR ------------------ #
df = pd.DataFrame(registros)

print("\nFilas totales (con posibles duplicados):", len(df))

# Quitamos duplicados por URL del anuncio
if "url_barco" in df.columns:
    df = df.drop_duplicates(subset=["url_barco"])

print("Filas tras eliminar duplicados por URL:", len(df))
print(df.head())

df.to_excel(
    "clickandboat_coruña_2026-07-24_26_sin_duplicados.xlsx",
    index=False
)
print("Excel guardado como 'clickandboat_coruña_2026-07-24_26_sin_duplicados.xlsx'")



=== PÁGINA 1: https://www.clickandboat.com/es/alquiler-barcos/busqueda?where=La%20Coru%C3%B1a,%20Espa%C3%B1a&DateDebut=2026-07-24&DateFin=2026-07-26&ProduitTypeId=&withCaptain= ===
   Tarjetas encontradas: 18

=== PÁGINA 2: https://www.clickandboat.com/es/alquiler-barcos/busqueda?where=La%20Coru%C3%B1a,%20Espa%C3%B1a&DateDebut=2026-07-24&DateFin=2026-07-26&ProduitTypeId=&withCaptain=&_page=2 ===
   Tarjetas encontradas: 18

=== PÁGINA 3: https://www.clickandboat.com/es/alquiler-barcos/busqueda?where=La%20Coru%C3%B1a,%20Espa%C3%B1a&DateDebut=2026-07-24&DateFin=2026-07-26&ProduitTypeId=&withCaptain=&_page=3 ===
   Tarjetas encontradas: 3

Filas totales (con posibles duplicados): 39
Filas tras eliminar duplicados por URL: 39
   pagina                                          url_barco  \
0       1  https://www.clickandboat.com/es/alquiler-barco...   
1       1  https://www.clickandboat.com/es/alquiler-barco...   
2       1  https://www.clickandboat.com/es/alquiler-barco...   
3       1  

In [10]:
#  SCRAPPING CLIKANDBOAT - IBIZA - JULIO 2026
#  -*- coding: utf-8 -*-
"""
Scraping Click&Boat (Ibiza, 24–26 julio 2026, páginas 1–44)
Extrae: nombre del barco, tipo de barco, precio, zona, rating,
capacidad de personas, patrón (sí/no/opcional), eslora (m) y URL del anuncio.
"""

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd

# ------------------ CONFIGURACIÓN SELENIUM ------------------ #
options = webdriver.ChromeOptions()
options.add_argument("--headless")          # comenta esta línea si quieres ver el navegador
options.add_argument("--window-size=1920,1080")

driver = webdriver.Chrome(options=options)  # necesitas chromedriver en el PATH

# ------------------ PARÁMETROS DE BÚSQUEDA (IBIZA) ------------------ #
BASE_URL = (
    "https://www.clickandboat.com/es/alquiler-barcos/busqueda"
    "?dateFrom=2026-07-24"
    "&dateTo=2026-07-26"
    "&where=Ibiza,%20Espa%C3%B1a"
)
# Páginas:
# 1 -> BASE_URL
# 2..44 -> BASE_URL + "&_page={page}"

BOAT_TYPES = {
    "Velero", "Lancha", "Catamarán", "Catamaran", "Neumática", "Neumatica",
    "Yate", "Moto de agua", "Goleta", "Casa flotante",
    "Barco sin licencia", "Bote sin licencia"
}

registros = []

# ------------------ RECORRER PÁGINAS ------------------ #
for page in range(1, 45):  # 1 a 44
    if page == 1:
        url = BASE_URL
    else:
        url = BASE_URL + f"&_page={page}"

    print(f"\n=== PÁGINA {page}: {url} ===")
    driver.get(url)

    wait = WebDriverWait(driver, 20)

    try:
        wait.until(
            EC.presence_of_all_elements_located(
                (By.CSS_SELECTOR, "app-extended-boat-card.product__card.extendedBoatCard")
            )
        )
    except Exception as e:
        print(f"   No se encontraron tarjetas en la página {page}: {e}")
        continue

    cards = driver.find_elements(
        By.CSS_SELECTOR, "app-extended-boat-card.product__card.extendedBoatCard"
    )
    print(f"   Tarjetas encontradas: {len(cards)}")

    for card in cards:
        # --------- URL del anuncio --------- #
        try:
            link_el = card.find_element(
                By.CSS_SELECTOR, "a.extendedBoatCard__link"
            )
            url_barco = link_el.get_attribute("href")
        except Exception:
            url_barco = ""

        # --------- Nombre del barco --------- #
        try:
            nombre = card.find_element(
                By.CSS_SELECTOR, ".extendedBoatCard__label"
            ).text.strip()
        except Exception:
            nombre = ""

        # --------- Tipo de barco y patrón --------- #
        tipo = ""
        patron = ""
        try:
            highlights = card.find_elements(
                By.CSS_SELECTOR,
                "ul.extendedBoatCard__highlights li.highlight"
            )
            for h in highlights:
                txt = h.text.strip()
                if txt in BOAT_TYPES and not tipo:
                    tipo = txt
                if "patrón" in txt.lower():
                    patron = txt
        except Exception:
            pass

        # --------- Precio por día --------- #
        try:
            precio_raw = card.find_element(
                By.CSS_SELECTOR, "span.pricePanel__price"
            ).text
            precio = precio_raw.replace("\xa0", " ").strip()
        except Exception:
            precio = ""

        # --------- Zona / localización --------- #
        try:
            zona = card.find_element(
                By.CSS_SELECTOR, ".extendedBoatCard__localization"
            ).text.strip()
        except Exception:
            zona = ""

        # --------- Rating (span.rate__avg) --------- #
        try:
            rating_raw = card.find_element(
                By.CSS_SELECTOR, ".extendedBoatCard__rate span.rate__avg"
            ).text.strip()
            rating = rating_raw.replace(",", ".")
        except Exception:
            rating = ""

        # --------- Capacidad personas y eslora --------- #
        capacidad_personas = ""
        eslora_m = ""
        try:
            caps = card.find_elements(
                By.CSS_SELECTOR, ".extendedBoatCard__capacities span"
            )
            for c in caps:
                txt = c.text.strip()
                if "persona" in txt.lower():
                    capacidad_personas = txt
                elif " m" in txt:
                    eslora_m = txt
        except Exception:
            pass

        registros.append(
            {
                "pagina": page,
                "url_barco": url_barco,
                "nombre_barco": nombre,
                "tipo_barco": tipo,
                "precio_dia": precio,
                "zona": zona,
                "rating": rating,
                "capacidad_personas": capacidad_personas,
                "patron": patron,
                "eslora_m": eslora_m,
            }
        )

# cerramos navegador
driver.quit()

# ------------------ CREAR DATAFRAME, QUITAR DUPLICADOS Y GUARDAR ------------------ #
df = pd.DataFrame(registros)

print("\nFilas totales (con posibles duplicados):", len(df))

# Quitamos duplicados por URL del anuncio
if "url_barco" in df.columns:
    df = df.drop_duplicates(subset=["url_barco"])

print("Filas tras eliminar duplicados por URL:", len(df))
print(df.head())

df.to_excel(
    "clickandboat_ibiza_2026-07-24_26_sin_duplicados.xlsx",
    index=False
)
print("Excel guardado como 'clickandboat_ibiza_2026-07-24_26_sin_duplicados.xlsx'")



=== PÁGINA 1: https://www.clickandboat.com/es/alquiler-barcos/busqueda?dateFrom=2026-07-24&dateTo=2026-07-26&where=Ibiza,%20Espa%C3%B1a ===
   Tarjetas encontradas: 18

=== PÁGINA 2: https://www.clickandboat.com/es/alquiler-barcos/busqueda?dateFrom=2026-07-24&dateTo=2026-07-26&where=Ibiza,%20Espa%C3%B1a&_page=2 ===
   Tarjetas encontradas: 18

=== PÁGINA 3: https://www.clickandboat.com/es/alquiler-barcos/busqueda?dateFrom=2026-07-24&dateTo=2026-07-26&where=Ibiza,%20Espa%C3%B1a&_page=3 ===
   Tarjetas encontradas: 18

=== PÁGINA 4: https://www.clickandboat.com/es/alquiler-barcos/busqueda?dateFrom=2026-07-24&dateTo=2026-07-26&where=Ibiza,%20Espa%C3%B1a&_page=4 ===
   Tarjetas encontradas: 18

=== PÁGINA 5: https://www.clickandboat.com/es/alquiler-barcos/busqueda?dateFrom=2026-07-24&dateTo=2026-07-26&where=Ibiza,%20Espa%C3%B1a&_page=5 ===
   Tarjetas encontradas: 18

=== PÁGINA 6: https://www.clickandboat.com/es/alquiler-barcos/busqueda?dateFrom=2026-07-24&dateTo=2026-07-26&where=Ibiza,%

In [15]:
#codigo para unir los ficheros excel generados en uno único solo y que se genera une columna con el nombre del fichero del que proviene cada fila   
#sin especificar ruta, los ficheros deben estar en el mismo directorio que este script
import pandas as pd
import glob
# Obtener la lista de todos los archivos Excel en el directorio actual
excel_files = glob.glob("*.xlsx")   
# Leer y concatenar todos los archivos en un solo DataFrame
df_list = []
for file in excel_files:
    df_temp = pd.read_excel(file)
    df_temp['origen_fichero'] = file  # Añadir columna con el nombre del fichero
    df_list.append(df_temp)
df_combined = pd.concat(df_list, ignore_index=True)
# Guardar el DataFrame combinado en un nuevo archivo Excel  
df_combined.to_excel("clickandboat_coruña_malaga_ibiza_2026-07-24_26.xlsx", index=False)
print("Excel combinado guardado como 'clickandboat_todos_los_destinos_2026-07-24_26.xlsx'")



Excel combinado guardado como 'clickandboat_todos_los_destinos_2026-07-24_26.xlsx'
